In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# URL do dataset HateBR no GitHub
DATASET_URL = 'https://raw.githubusercontent.com/franciellevargas/HateBR/main/dataset/HateBR.csv'

# 1. Carregar o dataset
print("Carregando o dataset...")
try:
    df = pd.read_csv(DATASET_URL)
    print("Dataset carregado com sucesso!")
    # Mostra as primeiras linhas e a distribuição das classes
    print(df.head())
    print("\nDistribuição das classes:")
    print(df['label_final'].value_counts(normalize=True))
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    exit()

# 2. Pré-processamento e Definição das variáveis
print("\nIniciando pré-processamento...")
# Para este MVP, a única limpeza será converter para minúsculas.
# O TfidfVectorizer já lida com muita coisa.
X = df['comentario'].str.lower()
y = df['label_final']

# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

# 4. Vetorização do texto usando TF-IDF
print("Vetorizando o texto...")
vectorizer = TfidfVectorizer(max_features=5000) # Usamos as 5000 palavras mais relevantes

# Aprende o vocabulário com os dados de treino e transforma os dados de treino
X_train_vect = vectorizer.fit_transform(X_train)

# Apenas transforma os dados de teste com o vocabulário já aprendido
X_test_vect = vectorizer.transform(X_test)

# 5. Treinamento do modelo de Regressão Logística
print("Treinando o modelo de classificação...")
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vect, y_train)
print("Modelo treinado com sucesso!")

# 6. Avaliação do modelo
print("\nAvaliando o modelo nos dados de teste...")
y_pred = model.predict(X_test_vect)

print("\nAcurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Não Odioso', 'Odioso']))

# Agora que o modelo está treinado, podemos usá-lo.
# Os objetos que precisamos salvar/usar para novas previsões são:
# - `model` (o classificador)
# - `vectorizer` (o vetorizador)

print("\n--- MVP PRONTO PARA USO ---")

Carregando o dataset...
Dataset carregado com sucesso!
   id                                         comentario  anotator1  \
0   1                                       Mais um lixo          1   
1   2                    Essa nao tem vergonha na cara!!          1   
2   3                     Essa mulher é doente.pilantra!          1   
3   4                                Comunista safada...          1   
4   5  Vagabunda. Comunista. Mentirosa. O povo chilen...          1   

   anotator2  anotator3  label_final  \
0          1          1            1   
1          1          1            1   
2          1          1            1   
3          1          1            1   
4          1          1            1   

                                 links_post    account_post  
0  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
1  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
2  https://www.instagram.com/p/B2uThqdH9xI/  Carla Zambelli  
3  https://www.instagram.com/

In [ ]:
def avaliar_toxicidade(comentario: str, model, vectorizer) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    # 1. Aplicar o mesmo pré-processamento (minúsculas)
    comentario_processado = comentario.lower()
    
    # 2. Vetorizar o comentário usando o vetorizador JÁ TREINADO
    comentario_vect = vectorizer.transform([comentario_processado])
    
    # 3. Fazer a predição
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    
    # A probabilidade de ser discurso de ódio é a probabilidade da classe "1"
    prob_odio = probabilidades[0][1]
    
    if predicao[0] == 1:
        classificacao = "Discurso de Ódio"
    else:
        classificacao = "Não é Discurso de Ódio"
        
    return {
        "classificacao": classificacao,
        "nivel_toxicidade": f"{prob_odio:.2%}" # Formata como porcentagem
    }

In [33]:
# --- EXEMPLOS DE USO ---
print("\n--- Testando o MVP com novos comentários ---")

# Exemplo 1: Comentário potencialmente tóxico
comentario1 = "Esses políticos são todos uns bandidos, tinham que sumir do mapa!"
resultado1 = avaliar_toxicidade(comentario1, model, vectorizer)
print(f"Comentário: '{comentario1}'")
print(f"Resultado: {resultado1}\n")

# Exemplo 2: Comentário neutro
comentario2 = "O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado."
resultado2 = avaliar_toxicidade(comentario2, model, vectorizer)
print(f"Comentário: '{comentario2}'")
print(f"Resultado: {resultado2}\n")

# Exemplo 3: Comentário inserido pelo usuário
'''print("Digite um comentário para ser analisado (ou 'sair' para terminar):")
while True:
    meu_comentario = input("> ")
    if meu_comentario.lower() == 'sair':
        break
    resultado = avaliar_toxicidade(meu_comentario, model, vectorizer)
    print(f"Resultado: {resultado}\n")'''


--- Testando o MVP com novos comentários ---
Comentário: 'Esses políticos são todos uns bandidos, tinham que sumir do mapa!'
Resultado: {'classificacao': 'Discurso de Ódio', 'nivel_toxicidade': '88.90%'}

Comentário: 'O jogo de futebol ontem foi muito emocionante, gostei bastante do resultado.'
Resultado: {'classificacao': 'Não é Discurso de Ódio', 'nivel_toxicidade': '35.25%'}



'print("Digite um comentário para ser analisado (ou \'sair\' para terminar):")\nwhile True:\n    meu_comentario = input("> ")\n    if meu_comentario.lower() == \'sair\':\n        break\n    resultado = avaliar_toxicidade(meu_comentario, model, vectorizer)\n    print(f"Resultado: {resultado}\n")'

In [34]:
# (Após o bloco de treinamento e avaliação do modelo...)
import joblib

print("\nSalvando o modelo e o vetorizador em arquivos...")

# Salva o modelo treinado
joblib.dump(model, 'modelo_odio.joblib')

# Salva o vetorizador
joblib.dump(vectorizer, 'vetorizador_odio.joblib')

print("Modelo e vetorizador salvos com sucesso!")


Salvando o modelo e o vetorizador em arquivos...
Modelo e vetorizador salvos com sucesso!


In [ ]:
import tweepy
import joblib
import os
import re

# --- CONFIGURAÇÃO ---
# Carregue o Bearer Token. É uma boa prática usar variáveis de ambiente.
# Para testar, você pode simplesmente colocar a string aqui:
# BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAHVr3wEAAAAAp2KKTt18ViS4Iy%2FseFXPFWMyA%2FA%3D07WZY4kChG6WCj7FJDMTCzimB1a2tKADe5HPzkgos7FR7UdLNM"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAHVr3wEAAAAAp2KKTt18ViS4Iy%2FseFXPFWMyA%2FA%3D07WZY4kChG6WCj7FJDMTCzimB1a2tKADe5HPzkgos7FR7UdLNM"

if not BEARER_TOKEN:
    print("ERRO: O Bearer Token não foi encontrado.")
    print("Por favor, defina a variável de ambiente TWITTER_BEARER_TOKEN ou insira-a diretamente no código.")
    exit()

# Carregar o modelo e o vetorizador salvos
try:
    model = joblib.load('modelo_odio.joblib')
    vectorizer = joblib.load('vetorizador_odio.joblib')
    print("Modelo e vetorizador carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivos 'modelo_odio.joblib' ou 'vetorizador_odio.joblib' não encontrados.")
    print("Certifique-se de executar o script de treinamento primeiro.")
    exit()

# --- FUNÇÕES ---

def avaliar_toxicidade_x(comentario: str) -> dict:
    """
    Recebe um comentário e retorna a classificação de toxicidade
    e a probabilidade de ser discurso de ódio.
    """
    comentario_processado = comentario.lower()
    comentario_vect = vectorizer.transform([comentario_processado])
    predicao = model.predict(comentario_vect)
    probabilidades = model.predict_proba(comentario_vect)
    prob_odio = probabilidades[0][1]
    
    return {
        "eh_odio": predicao[0] == 1,
        "nivel_toxicidade": prob_odio
    }

def buscar_tweets_usuario(client, username: str, count: int = 20):
    """
    Busca os tweets mais recentes de um usuário.
    """
    try:
        # 1. Encontrar o ID do usuário a partir do nome de usuário
        user_response = client.get_user(username=username)
        if not user_response.data:
            print(f"Usuário '{username}' não encontrado.")
            return None
        user_id = user_response.data.id

        # 2. Buscar os tweets, excluindo respostas e retweets
        tweets_response = client.get_users_tweets(
            id=user_id,
            max_results=count,
            exclude=['replies', 'retweets'],
            tweet_fields=['text']
        )
        return tweets_response.data
    except Exception as e:
        print(f"Ocorreu um erro ao buscar tweets de @{username}: {e}")
        return None

def limpar_tweet(texto: str) -> str:
    """
    Remove links, menções e outros ruídos do texto do tweet.
    """
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@\w+', '', texto)
    texto = re.sub(r'#', '', texto)
    texto = texto.strip()
    return texto

# --- EXECUÇÃO PRINCIPAL ---

def main():
    print("\n--- INICIANDO ANÁLISE DE TOXICIDADE NO X ---")
    
    # Conectar à API
    try:
        client = tweepy.Client(bearer_token=BEARER_TOKEN)
        print("Conexão com a API do X estabelecida.")
    except Exception as e:
        print(f"Erro ao conectar com a API do X: {e}")
        return

    # Alvos da análise
    alvos = ["felipeneto", "casimiro"]
    num_tweets_para_analisar = 20 # Pode aumentar ou diminuir

    for usuario in alvos:
        print(f"\n--- Analisando @{usuario} ---")
        
        tweets = buscar_tweets_usuario(client, usuario, count=num_tweets_para_analisar)
        
        if not tweets:
            continue

        total_toxicidade = 0
        tweets_odiosos = 0
        
        for i, tweet in enumerate(tweets):
            texto_limpo = limpar_tweet(tweet.text)
            if not texto_limpo:
                continue

            resultado = avaliar_toxicidade_x(texto_limpo)
            total_toxicidade += resultado["nivel_toxicidade"]
            if resultado["eh_odio"]:
                tweets_odiosos += 1
                # Descomente a linha abaixo para ver os tweets classificados como odiosos
                # print(f"  [ODIOSO DETECTADO] Prob: {resultado['nivel_toxicidade']:.2%} | Tweet: {texto_limpo[:50]}...")

        # Relatório final para o usuário
        num_analisados = len(tweets)
        media_toxicidade = (total_toxicidade / num_analisados) if num_analisados > 0 else 0
        
        print(f"Relatório Final para @{usuario}:")
        print(f"  - Tweets recentes analisados: {num_analisados}")
        print(f"  - Tweets classificados como discurso de ódio: {tweets_odiosos}")
        print(f"  - Nível médio de toxicidade: {media_toxicidade:.2%}")

if __name__ == "__main__":
    main()

Modelo e vetorizador carregados com sucesso.

--- INICIANDO ANÁLISE DE TOXICIDADE NO X ---
Conexão com a API do X estabelecida.

--- Analisando @felipeneto ---
Ocorreu um erro ao buscar tweets de @felipeneto: 429 Too Many Requests
Too Many Requests

--- Analisando @casimiro ---
Ocorreu um erro ao buscar tweets de @casimiro: 429 Too Many Requests
Too Many Requests
